In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
import os

import flax
from flax import jax_utils
import jax
import jax.numpy as jnp
import numpy as np
import optax
import torch
import tensorflow as tf
import tqdm

import init2winit
import fastmri

import i2w

from fastmri.models import unet as t_unet
from fastmri.pl_modules import data_module
from fastmri.pl_modules import unet_module
from fastmri.data.transforms import UnetDataTransform
from fastmri.data.subsample import create_mask_for_mask_type

from init2winit.model_lib import unet as f_unet
from init2winit.dataset_lib import fastmri_dataset
from init2winit.dataset_lib import data_utils
from init2winit.optimizer_lib import optimizers

jax.devices()

Extension horovod.torch has not been built: /opt/conda/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.


[GpuDevice(id=0, process_index=0),
 GpuDevice(id=1, process_index=0),
 GpuDevice(id=2, process_index=0),
 GpuDevice(id=3, process_index=0),
 GpuDevice(id=4, process_index=0),
 GpuDevice(id=5, process_index=0),
 GpuDevice(id=6, process_index=0),
 GpuDevice(id=7, process_index=0)]

# Data pipeline

In [2]:
data_path = pathlib.Path('../../')
challenge = 'singlecoil'
center_fractions = [0.08]
accelerations = [4]
mask_type = 'random'  # Should be `random`, but tying out without
test_split = 'test'

# NOTE: We test without masking

## `fastmri`

In [3]:
mask = create_mask_for_mask_type(mask_type, center_fractions, accelerations)
mask = None

train_transform = UnetDataTransform(challenge, mask_func=mask, use_seed=False)
val_transform = UnetDataTransform(challenge, mask_func=mask)
test_transform = UnetDataTransform(challenge)

dm = data_module.FastMriDataModule(
    data_path=data_path,
    challenge=challenge,
    train_transform=train_transform,
    val_transform=val_transform,
    test_transform=test_transform,
    test_split=test_split,
    batch_size=1,
    num_workers=4,
)

dl = iter(dm._create_data_loader(train_transform, data_partition='train'))

## `init2winit`

In [4]:
for i, t_batch in enumerate(tqdm.tqdm(dl)):
    inputs = i2w.get_slice(os.path.join('../../singlecoil_train', t_batch.fname[0]), t_batch.slice_num[0])
    f_batch = fastmri_dataset._process_example(*inputs, tf.cast(jax.random.PRNGKey(0), tf.int64))
    
    input_delta = np.mean(np.abs(f_batch['inputs'].numpy() - t_batch.image.numpy().squeeze()))
    output_delta = np.mean(np.abs(f_batch['targets'].numpy() - t_batch.target.numpy().squeeze()))
    vol_delta = f_batch['volume_max'] - t_batch.max_value.numpy().squeeze()
    
    print(i, input_delta, output_delta, vol_delta)

  0%|                                                                                                                                                                          | 5/34742 [00:06<9:05:15,  1.06it/s]

0 5.5568657e-06 5.494738e-06 0.0
1 3.2723403e-06 3.2581686e-06 0.0
2 3.37865e-06 3.3869771e-06 0.0
3 3.7209556e-06 3.7376851e-06 0.0
4 3.985859e-06 4.0059226e-06 0.0
5 4.1342105e-06 4.155034e-06 0.0
6 4.3241125e-06 4.3441555e-06 0.0
7 4.3946766e-06 4.4219473e-06 0.0
8 4.2313254e-06 4.2744696e-06 0.0


  0%|                                                                                                                                                                         | 15/34742 [00:06<2:06:12,  4.59it/s]

9 4.1522803e-06 4.189665e-06 0.0
10 3.9324304e-06 3.9652023e-06 0.0
11 3.543296e-06 3.5699472e-06 0.0
12 3.3664003e-06 3.3918848e-06 0.0
13 3.3732981e-06 3.400605e-06 0.0
14 3.2851274e-06 3.3055883e-06 0.0
15 3.3284064e-06 3.3419142e-06 0.0
16 3.2805715e-06 3.2824425e-06 0.0
17 3.526097e-06 3.5293235e-06 0.0
18 3.3587653e-06 3.3668057e-06 0.0


  0%|                                                                                                                                                                           | 25/34742 [00:06<56:20, 10.27it/s]

...

  1%|██▍                                                                                                                                                                       | 505/34742 [00:17<12:33, 45.43it/s]

496 4.0050368e-06 4.114091e-06 0.0
497 4.8999436e-06 4.840256e-06 0.0
498 2.3191592e-06 2.3225919e-06 0.0
499 2.7739356e-06 2.7966694e-06 0.0
500 3.2756027e-06 3.3144117e-06 0.0
501 3.571808e-06 3.6151387e-06 0.0
502 3.8509625e-06 3.8966377e-06 0.0
503 4.0535388e-06 4.0994046e-06 0.0
504 4.12187e-06 4.1682188e-06 0.0


  1%|██▍                                                                                                                                                                       | 509/34742 [00:17<19:25, 29.37it/s]


505 4.329506e-06 4.3729005e-06 0.0
506 4.2712554e-06 4.304161e-06 0.0
507 4.444456e-06 4.488896e-06 0.0
508 4.41099e-06 4.4674675e-06 0.0


KeyboardInterrupt: 